In [ ]:
import yaml
from pathlib import Path

import tifffile
import torch
import numpy as np
import matplotlib.pyplot as plt

from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity

from metrics import scale_invariant_psnr

from tqdm import tqdm

from predict import predict_image, load_model_and_config

In [ ]:

def calculate_metrics_for_folder(
    folder_path, model,
    input_stats,
    target_stats,
    device="cuda"
):
    folder_path = Path(folder_path)
    image_files = sorted(list(folder_path.glob('*.tif')))

    si_psnr_values = []
    psnr_values = []
    ssim_values = []

    for image_path in tqdm(image_files):
        # Load image (expecting HR, LR pair)
        image = tifffile.imread(image_path)
        hr, lr = image

        result = predict_image(lr, model, input_stats, target_stats)

        si_psnr = scale_invariant_psnr(hr, result)
        psnr = peak_signal_noise_ratio(hr, result, data_range=hr.max() - hr.min())
        ssim = structural_similarity(hr, result, data_range=hr.max() - hr.min())

        si_psnr_values.append(0 if np.isnan(si_psnr) else si_psnr)
        psnr_values.append(0 if np.isnan(psnr) else psnr)
        ssim_values.append(0 if np.isnan(ssim) else ssim)

    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)
    avg_si_psnr = np.mean(si_psnr_values)

    return {'psnr': avg_psnr, 'ssim': avg_ssim, 'si_psnr': avg_si_psnr}

In [ ]:
models = Path('../experiments/').glob('*')

results = []
for model in models:
    model, config = load_model_and_config(model)
    val_folder = config['datasets']['val']['dataroot']
    input_stats = config['datasets']['train']['input_mean'], \
        config['datasets']['train']['input_std']
    target_stats = config['datasets']['train']['target_mean'], \
        config['datasets']['train']['target_std']
    result = calculate_metrics_for_folder(
        val_folder,
        model,
        input_stats,
        target_stats
    )
    result['name'] = type
    results.append(result)

import pandas as pd

df = pd.DataFrame.from_dict(results)
df = df[['name', 'si_psnr', 'ssim']]
df = df.T
df.columns = df.iloc[0]
df = df.drop('name')
df = df.sort_index()
df

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(results)
df = df[['name', 'si_psnr', 'ssim']]
df = df.T
df.columns = df.iloc[0]
df = df.drop('name')
df = df.sort_index()
df

In [ ]:
def visualize_prediction(image_path, model, image_mean, image_std, target_mean, target_std, device="cuda"):
    """Visualize prediction results for a single image."""
    if device == "cuda" and not torch.cuda.is_available():
        device = "cpu"

    # Load image
    image = tifffile.imread(image_path)
    if len(image.shape) == 3 and image.shape[0] == 2:
        hr, lr = image
    else:
        print(f"Warning: {image_path} doesn't have expected HR/LR pair format")
        return

    # Predict
    result = predict_image(model, lr, image_mean, image_std, target_mean, target_std, device)

    # Calculate metrics
    si_psnr = scale_invariant_psnr(hr, result)
    psnr = peak_signal_noise_ratio(hr, result, data_range=hr.max() - hr.min())
    ssim = structural_similarity(hr, result, data_range=hr.max() - hr.min())

    # Visualization
    fig, ax = plt.subplots(1, 3, figsize=(12, 3))
    ax[0].imshow(lr)
    ax[0].set_title('input')
    ax[1].imshow(result)
    ax[1].set_title('predicted')
    ax[1].text(
        5, 20, f'PSNR: {psnr:.2f}\nSI-PSNR: {si_psnr:.2f}\nSSIM: {ssim:.4f}', color='white',
        bbox=dict(facecolor='black', alpha=0)
    )
    ax[2].imshow(hr)
    ax[2].set_title('target')

    # Histograms
    fig, ax = plt.subplots(1, 3, figsize=(12, 2))
    ax[0].hist(lr.flatten(), bins=100)
    ax[0].set_title('Input histogram')
    ax[1].hist(result.flatten(), bins=100)
    ax[1].set_title('Predicted histogram')
    ax[2].hist(hr.flatten(), bins=100)
    ax[2].set_title('Target histogram')

    plt.show()

    return {'psnr': psnr, 'ssim': ssim, 'si_psnr': si_psnr}


In [ ]:
def visualize_prediction(image_path, model, image_mean, image_std, target_mean, target_std, device="cuda"):
    """Visualize prediction results for a single image."""
    if device == "cuda" and not torch.cuda.is_available():
        device = "cpu"

    # Load image
    image = tifffile.imread(image_path)
    if len(image.shape) == 3 and image.shape[0] == 2:
        hr, lr = image
    else:
        print(f"Warning: {image_path} doesn't have expected HR/LR pair format")
        return

    # Predict
    result = predict_image(model, lr, image_mean, image_std, target_mean, target_std, device)

    # Calculate metrics
    si_psnr = scale_invariant_psnr(hr, result)
    psnr = peak_signal_noise_ratio(hr, result, data_range=hr.max() - hr.min())
    ssim = structural_similarity(hr, result, data_range=hr.max() - hr.min())

    # Visualization
    fig, ax = plt.subplots(1, 3, figsize=(12, 3))
    ax[0].imshow(lr)
    ax[0].set_title('input')
    ax[1].imshow(result)
    ax[1].set_title('predicted')
    ax[1].text(
        5, 20, f'PSNR: {psnr:.2f}\nSI-PSNR: {si_psnr:.2f}\nSSIM: {ssim:.4f}', color='white',
        bbox=dict(facecolor='black', alpha=0)
    )
    ax[2].imshow(hr)
    ax[2].set_title('target')

    # Histograms
    fig, ax = plt.subplots(1, 3, figsize=(12, 2))
    ax[0].hist(lr.flatten(), bins=100)
    ax[0].set_title('Input histogram')
    ax[1].hist(result.flatten(), bins=100)
    ax[1].set_title('Predicted histogram')
    ax[2].hist(hr.flatten(), bins=100)
    ax[2].set_title('Target histogram')

    plt.show()

    return {'psnr': psnr, 'ssim': ssim, 'si_psnr': si_psnr}

import random
config_path = Path(f'../options/train/RCAN/')

results = []
for folder in Path('../experiments/').glob('*'):
    type = folder.name.split('_')[-2]
    config = config_path / f"train_rcan_biosr_{type.lower()}.yml"
    checkpoint = folder / "models" / "net_g_latest.pth"
    if not checkpoint.exists():
        checkpoint = sorted((folder / "models").glob('*.pth'))[-1]

    if config.exists() and checkpoint.exists():
        print(f"\nVisualizing predictions for {type} model:")
        config = yaml.load(open(config, 'r'), Loader=yaml.FullLoader)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = load_model(checkpoint, config['network_g'], device)
        val_folder = Path(config['datasets']['val']['dataroot'])

        # Get 5 random validation images
        val_images = list(val_folder.glob('*.tif'))
        sample_images = random.sample(val_images, min(5, len(val_images)))

        for img_path in sample_images:
            print(f"\nPrediction for {img_path.name}:")
            metrics = visualize_prediction(
                img_path,
                model,
                image_mean=config['datasets']['train']['input_mean'],
                image_std=config['datasets']['train']['input_std'],
                target_mean=config['datasets']['train']['target_mean'],
                target_std=config['datasets']['train']['target_std'],
                device=device
            )
            if metrics:
                results.append({**metrics, 'name': type})